# Linear Classification

In [1]:
# Import torch and other required modules
import torch
import numpy as np

In [2]:
# Additional imports
from data import cifar_10_data

## Load CIFAR-10 Dataset

Training Data

In [5]:
data_dict= cifar_10_data.get_training_data()

In [6]:
# Get image data
data_input= data_dict["data"]
data_input.shape

(3072, 10000)

We transpose the input matrix such that each row represents 1 sample (by default, the CIFAR-10 dataset has its columns as data samples)

In [7]:
data_input= data_input.T.astype(float)

In [8]:
# Get image labels
data_label= data_dict["labels"].astype(float)
data_label

array([6., 9., 9., ..., 1., 1., 5.])

Test Data

In [9]:
data_test_dict= cifar_10_data.get_test_data()

## tensor.from_numpy()
Create a Tensor from a numpy array. An interesting fact about this function is that the created Tensor and the underlying numpy array share the same memory. Thus changing either reflects on the other.

Convert to PyTorch Tensor

Convertion of input images (in numpy array) to tensor

In [10]:
input_ts= torch.from_numpy(data_input)
# Convert to flloat
input_ts= input_ts.float()
input_ts

tensor([[ 59.,  43.,  50.,  ..., 140.,  84.,  72.],
        [154., 126., 105.,  ..., 139., 142., 144.],
        [255., 253., 253.,  ...,  83.,  83.,  84.],
        ...,
        [ 71.,  60.,  74.,  ...,  68.,  69.,  68.],
        [250., 254., 211.,  ..., 215., 255., 254.],
        [ 62.,  61.,  60.,  ..., 130., 130., 131.]])

Conversion of labels to tensor

In [11]:
label_ts= torch.from_numpy(data_label)
# COnvert to float
label_ts= label_ts.float()
label_ts

tensor([6., 9., 9.,  ..., 1., 1., 5.])

Conversion of test data to Tensors

In [12]:
test_input_ts= torch.from_numpy(data_test_dict["data"].T).float()
test_label_ts= torch.from_numpy(data_test_dict["labels"]).float()

## torch.utils.data.TensorData
Create dataset for use with DataLoader

In [13]:
from torch.utils.data import TensorDataset

Create a TensorDataset object from training data

In [14]:
# Example 1 - working
train_ds= TensorDataset(input_ts, label_ts)

Create a TensorDataset from test data

In [15]:
# Example 2 - working
test_ds= TensorDataset(test_input_ts, test_label_ts)

This function created a TensorDataset object which can be supplied as a dataset to a DataLoader. Details of the DataLoader would be provided below.

## torch.utils.data.DataLoader

Create a DataLoader object through which PyTorch provides functionality such as modifying data loading order and data batching.

In [16]:
from torch.utils.data import DataLoader

Create a DataLoader from training data. Setting the *batch_size* kwarg makes the DataLoader yield samples in batches (default is *1*). Setting the *shuffle* kwarg to *True* makes it possible to pick random samples from the dataset.

In [17]:
# Example 1 - working
train_dl= DataLoader(train_ds, batch_size=500, shuffle=True)

Creates a DataLoader for testing data.

In [18]:
# Example 2 - working
test_dl= DataLoader(test_ds)

## torch.nn.Linear

Create a linear model while initializing the weights and biases in the process.

In [19]:
import torch.nn as nn

Linear model to train. The first argument is the size of each input sample (3072 in our case), and the second is the size of each output sample (1 in our case)

In [20]:
model= nn.Linear(input_ts.shape[1], 10)

This function provides an easy way to define linear models and saves the stress of having to initialize weights.

In [21]:
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0091,  0.0171,  0.0170,  ...,  0.0023, -0.0026, -0.0157],
        [ 0.0024,  0.0070, -0.0008,  ...,  0.0139, -0.0042,  0.0069],
        [-0.0162, -0.0146, -0.0051,  ..., -0.0094,  0.0142, -0.0089],
        ...,
        [-0.0132,  0.0159,  0.0103,  ..., -0.0121, -0.0015,  0.0050],
        [ 0.0010, -0.0121,  0.0027,  ..., -0.0164, -0.0032, -0.0050],
        [ 0.0054, -0.0144,  0.0065,  ..., -0.0126,  0.0055, -0.0103]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0137, -0.0179, -0.0103,  0.0023, -0.0128,  0.0049, -0.0159,  0.0057,
        -0.0058, -0.0047], requires_grad=True)


In [22]:
# List of all parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.0091,  0.0171,  0.0170,  ...,  0.0023, -0.0026, -0.0157],
         [ 0.0024,  0.0070, -0.0008,  ...,  0.0139, -0.0042,  0.0069],
         [-0.0162, -0.0146, -0.0051,  ..., -0.0094,  0.0142, -0.0089],
         ...,
         [-0.0132,  0.0159,  0.0103,  ..., -0.0121, -0.0015,  0.0050],
         [ 0.0010, -0.0121,  0.0027,  ..., -0.0164, -0.0032, -0.0050],
         [ 0.0054, -0.0144,  0.0065,  ..., -0.0126,  0.0055, -0.0103]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0137, -0.0179, -0.0103,  0.0023, -0.0128,  0.0049, -0.0159,  0.0057,
         -0.0058, -0.0047], requires_grad=True)]

Compute output of the linear model for first 3 samples

In [42]:
pred_0= model(input_ts[0:3, :])

## torch.nn.MultiLabelMarginLoss
Hingeloss

In [51]:
hinge_loss= torch.nn.MultiLabelMarginLoss()

Compute loss for first 3 samples

In [57]:
label_ts[0]

tensor(6.)

In [65]:
pred_0

tensor([[  39.6937,  -34.0227,  142.6758,    4.5683,   89.1678,   42.3686,
          117.4220,   58.0179,   37.1425, -113.0736],
        [ -20.4629,  -54.7305,   43.7133,  -19.7414,  103.0743,   44.2047,
          136.7685,   26.7610,   49.3197, -119.6131],
        [  55.3449,  -57.1309,   30.9329,   53.3827,   82.5203,   36.6238,
           70.8297,   -8.7942,   96.8193,   13.7300]], grad_fn=<AddmmBackward>)

Formulate vector for incorrect labels

In [108]:
label_hinge= torch.from_numpy(-1 * np.ones(pred_0.shape))
label_hinge= label_hinge.to(torch.float32)
label_hinge

tensor([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]])

In [109]:
hinge_truth= torch.Tensor(label_ts[0:3])
hinge_truth.dtype

torch.float32

In [112]:
# set correct labels in each row
label_hinge[ :,0 ]= hinge_truth
label_hinge= label_hinge.to(torch.float64)

tensor([[ 6., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
        [ 9., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
        [ 9., -1., -1., -1., -1., -1., -1., -1., -1., -1.]],
       dtype=torch.float64)

In [118]:
input_hinge= torch.Tensor(pred_0)
input_hinge.to(dtype=torch.float64)

tensor([[  39.6937,  -34.0227,  142.6758,    4.5683,   89.1678,   42.3686,
          117.4220,   58.0179,   37.1425, -113.0736],
        [ -20.4629,  -54.7305,   43.7133,  -19.7414,  103.0743,   44.2047,
          136.7685,   26.7610,   49.3197, -119.6131],
        [  55.3449,  -57.1309,   30.9329,   53.3827,   82.5203,   36.6238,
           70.8297,   -8.7942,   96.8193,   13.7300]], dtype=torch.float64,
       grad_fn=<CopyBackwards>)

In [119]:
hinge_loss(input_hinge, label_hinge)

RuntimeError: expected scalar type Long but found Float